## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
from collections import defaultdict
import itertools

def Enumerator(start=0, initial=()):
    return defaultdict(itertools.count(start).__next__, initial)

In [7]:
import glob
import csv
from tqdm.auto import tqdm

id_enumerator = Enumerator()
row_dict = {}
left_id_set = set()
right_id_set = set()
rows_total = 1363 + 3226
clusters_total = 1300

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads//Amazon-GoogleProducts/Amazon.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['name'] = row.pop('title')
            row['source'] = 'google'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/Amazon-GoogleProducts/GoogleProducts.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'amazon'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [8]:
true_pair_set = set()

with open(f'{home_dir}/Downloads/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['idAmazon']]
        id_right = id_enumerator[row['idGoogleBase']]
        true_pair_set.add(tuple(sorted([id_left, id_right])))

len(true_pair_set)

1300

In [9]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict

cluster_mapping, cluster_dict = id_pairs_to_cluster_mapping_and_dict(true_pair_set)
len(cluster_mapping)

2404

In [10]:
len(cluster_dict)

1105

In [11]:
# TODO: deal with this difference
# from entity_embed.data_utils.utils import cluster_dict_to_id_pairs

# assert len(true_pair_set - cluster_dict_to_id_pairs(cluster_dict)) == 0

In [12]:
cluster_attr = 'cluster_id'
max_cluster_id = max(cluster_mapping.values())

for row_id, row in tqdm(row_dict.items()):
    try:
        row[cluster_attr] = cluster_mapping[row_id]
    except KeyError:
        row[cluster_attr] = max_cluster_id
        max_cluster_id += 1

In [13]:
[row_dict[row_id] for row_id in next(iter(true_pair_set))]

[{'id': 938,
  'description': 'improve your typing skills today! typing instructor deluxe has a progressive design that has been developed for over 19 years. typing instructor deluxe can provide the right lessons tests strengthening exercises practice material and typing games for your skill level. you can even build your own personal typing plan to focus on specific areas you would like to improve. if you think learning has to be all hard work and no fun think again! for beginning to advanced typists kids to adults typing instructor deluxe will motivate you to improve your typing speed and accuracy using a travel theme and exciting typing challenges.educates entertains and motivates: choose from many typing plans or build your ownnavigate easily and choose your typing materialnew! dynamic learning methodsave your results and reports to track progresslearn voice-touch typing (dictation)3 unique travel themescolorful photos and musicten exciting games300+ magazine articles',
  'manufact

## Preprocess

In [14]:
attr_list = ['name', 'description', 'manufacturer', 'price']

In [15]:
import unidecode
from entity_embed.data_utils.one_hot_encoders import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [16]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [17]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [18]:
attr_info_dict = {
    'name': {
        'is_multitoken': True,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'description': {
        'is_multitoken': True,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'manufacturer': {
        'is_multitoken': True,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,  # compute
        'max_str_len': None,  # compute
    }
}

In [19]:
from entity_embed import MultiSigLinkageEmbed

train_cluster_len = 200
valid_cluster_len = 200
ann_k = 100
sim_threshold = 0.3
use_mask = True
model = MultiSigLinkageEmbed(
    # data kwargs
    row_dict=row_dict,
    attr_info_dict=attr_info_dict,
    cluster_attr=cluster_attr,
    pos_pair_batch_size=45,
    neg_pair_batch_size=1225,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=clusters_total - valid_cluster_len - train_cluster_len,
    only_plural_clusters=True,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    random_seed=random_seed,
    # model kwargs
    use_mask=use_mask,
    ann_k=ann_k,
    sim_threshold=sim_threshold
)

16:04:41 INFO:For attr='name', computing actual alphabet and max_str_len
16:04:41 INFO:For attr='name', using actual_max_str_len=26
16:04:41 INFO:For attr='description', computing actual alphabet and max_str_len
16:04:41 INFO:actual_max_str_len=29 must be pair to enable NN pooling. Updating to 30
16:04:41 INFO:For attr='description', using actual_max_str_len=30
16:04:41 INFO:For attr='manufacturer', computing actual alphabet and max_str_len
16:04:41 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
16:04:41 INFO:For attr='manufacturer', using actual_max_str_len=16
16:04:41 INFO:For attr='price', computing actual alphabet and max_str_len
16:04:41 INFO:For attr='price', using actual_max_str_len=14


## Training

In [20]:
gpus = 1
max_epochs = 50
check_val_every_n_epoch = 1
early_stopping_monitor = 'valid_recall'
tb_log_dir = 'tb_logs'
tb_name = 'amzn-googl'
zero_weight = 0.1

model.fit(
    gpus=gpus,
    max_epochs=max_epochs,
    check_val_every_n_epoch=check_val_every_n_epoch,
    early_stopping_monitor=early_stopping_monitor,
    tb_log_dir=tb_log_dir,
    tb_name=tb_name,
    zero_weight=zero_weight
)

16:04:41 INFO:Fit model_sig_i=0, learning signature with attr_list=['name', 'description', 'manufacturer', 'price']
16:04:41 INFO:GPU available: True, used: True
16:04:41 INFO:TPU available: None, using: 0 TPU cores
16:04:41 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
16:04:41 INFO:Train pair count: 292
16:04:41 INFO:Valid pair count: 265
16:04:41 INFO:Test pair count: 998
16:04:43 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 3.3 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params


16:05:16 INFO:Fit model_sig_i=1, learning signature with attr_list=['manufacturer']
16:05:16 INFO:GPU available: True, used: True
16:05:16 INFO:TPU available: None, using: 0 TPU cores
16:05:16 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
16:05:16 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 640 K 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
640 K     Trainable params
0         Non-trainable params
640 K     Total params


In [21]:
for lt_module in model.lt_module_list:
    display(lt_module.get_signature_weights())

{'name': 0.5947720408439636,
 'description': 0.2792451083660126,
 'manufacturer': 0.011868284083902836,
 'price': 0.11411460489034653}

{'manufacturer': 1.0}

## Testing manually 

In [22]:
model.datamodule.setup(stage='test')

16:05:38 INFO:Train pair count: 292
16:05:38 INFO:Valid pair count: 265
16:05:38 INFO:Test pair count: 998


In [33]:
test_row_dict = model.datamodule.test_row_dict
test_multisig_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=model.datamodule.left_id_set,
    right_id_set=model.datamodule.right_id_set,
    batch_size=16
)
test_multisig_dict.keys()

dict_keys([('name', 'description', 'manufacturer', 'price'), ('manufacturer',)])

In [24]:
test_true_pair_set = model.datamodule.test_true_pair_set
len(test_true_pair_set)

834

In [35]:
%%time

from entity_embed import MultiSigANNLinkageIndex

ann_index = MultiSigANNLinkageIndex(
    multisig_dict_keys=model.multisig_dict_keys,
    embedding_size=model.embedding_size,
)
ann_index.insert_multisig_dict(test_multisig_dict)
ann_index.build()

CPU times: user 3.43 s, sys: 77.5 ms, total: 3.51 s
Wall time: 649 ms


In [36]:
%%time

found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    multisig_dict=test_multisig_dict,
)

CPU times: user 1.33 s, sys: 12.1 ms, total: 1.35 s
Wall time: 287 ms


In [37]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

109.2334200260078

In [38]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.004904732709924346, 0.988009592326139)

In [39]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

167177

In [40]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

10

In [41]:
cos_similarity = lambda a, b: np.dot(a, b)

In [42]:
test_multisig_dict.keys()

dict_keys([('name', 'description', 'manufacturer', 'price'), ('manufacturer',)])

In [46]:
test_left_vector_dict, test_right_vector_dict = \
    test_multisig_dict[('name', 'description', 'manufacturer', 'price')]

for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.29749808,
 {'id': 1009,
  'description': 'test and improve your memory is a personal trainer for the mind . for your brain to operate it is peak it needs regular exercise . keep it in shape',
  'manufacturer': 'tri synergy',
  'price': '29 . 99',
  'name': 'test and improve your memory ( dvd box )',
  'source': 'google',
  'cluster_id': 1009},
 {'id': 2180,
  'name': 'tri synergy inc test & improve your memory - scientific brain training',
  'description': 'engage challenge and train your memory as it is a crucial area of the brain that is important to your life . includes many games to have fun with various',
  'manufacturer': '',
  'price': '24 . 31',
  'source': 'amazon',
  'cluster_id': 1009})

(0.23020662,
 {'id': 1183,
  'description': 'create the perfect pet for your sims to train play with and love throughout a lifetime . pick everything from their paws to their personality . choose from dozens of',
  'manufacturer': 'aspyr media',
  'price': '34 . 99',
  'name': 'sims 2 pets expansion pack',
  'source': 'google',
  'cluster_id': 1183},
 {'id': 2255,
  'name': 'sims 2 pets for mac',
  'description': 'system requirements : requires the full version of the sims 2 for mac os x to play operating system : mac os x 10 . 3 . 9 or later',
  'manufacturer': '',
  'price': '34 . 99',
  'source': 'amazon',
  'cluster_id': 1183})

(0.05884345,
 {'id': 918,
  'description': 'mobi 70008 recam 4 . 6 camera monitoring and notification software surveillance software captures and analyzes images for advanced monitoring and recording of home or office ; program performs simultaneous',
  'manufacturer': 'mobi technologies inc .',
  'price': '129',
  'name': 'recam remote monitoring software',
  'source': 'google',
  'cluster_id': 918},
 {'id': 3921,
  'name': 'mobi - cam 70008 monitoring and notification software',
  'description': 'easily set - up and monitor your home or business observation system from anywhere notification of alarm and viewing via e - mail internet or cell phone multiple camera support',
  'manufacturer': 'mobi - cam',
  'price': '89 . 69',
  'source': 'amazon',
  'cluster_id': 918})

(0.0054139066,
 {'id': 845,
  'description': "iplaymusic ' s beginner guitar lessons is the first guitar learning program optimized for the mac and video ipod . the software takes advantage of ilife applications resident on today",
  'manufacturer': 'iplaymusic',
  'price': '49 . 99',
  'name': 'iplaymusic beginner guitar lessons for the mac and ipod',
  'source': 'google',
  'cluster_id': 845},
 {'id': 2324,
  'name': "wingnuts 2 : raina ' s revenge",
  'description': 'system requirements : mac os x 10 . 4 + g4 / g5 / intel 800 + mhz cpu 512 mb ram 32 mb video card 950 mb hard drive',
  'manufacturer': '',
  'price': '28 . 99',
  'source': 'amazon',
  'cluster_id': 845})

(0.25785547,
 {'id': 641,
  'description': "sims 2 : nightlife takes your sim into the night . explore all of your favorite after - dark activities as you discover your sims ' love lives or have",
  'manufacturer': 'aspyr media',
  'price': '34 . 99',
  'name': 'sims 2 nightlife expansion pack',
  'source': 'google',
  'cluster_id': 641},
 {'id': 1908,
  'name': 'aspyr media inc sims 2 nightlife',
  'description': "send your sims on an epic night out ! your sims are on the town hitting all the swanky hot spots . whether they ' re dancing until dawn romancing",
  'manufacturer': '',
  'price': '33 . 21',
  'source': 'amazon',
  'cluster_id': 641})

(0.16682643,
 {'id': 1124,
  'description': 'printmusic ! is simply the best entry level notation software . consider it a word processor for printing music notes instead of text . designed for the less demanding user',
  'manufacturer': 'makemusic !',
  'price': '69 . 99',
  'name': 'emedia print music 2006 win / mac',
  'source': 'google',
  'cluster_id': 1124},
 {'id': 3546,
  'name': 'make printmusic 2006 software music production software',
  'description': "printmusic 2006 - sheet music creation software - mac os x and windows designed for educators worship directors performing musicians composers and arrangers who don ' t require the advanced",
  'manufacturer': '',
  'price': '75 . 95',
  'source': 'amazon',
  'cluster_id': 1124})

(0.2806734,
 {'id': 529,
  'description': "zoo tycoon 2 : marine mania is a great new expansion where you ' ll help your zoo make a big splash ! add killer whales manta rays and other",
  'manufacturer': 'microsoft',
  'price': '19 . 99',
  'name': 'zoo tycoon 2 : marine mania expansion',
  'source': 'google',
  'cluster_id': 529},
 {'id': 2963,
  'name': 'zoo tycoon for windows',
  'description': 'everybody likes the zoo . and why not casual walks exotic animals and fried foods are good fun . managing a zoo of your very own is even better .',
  'manufacturer': '',
  'price': '25 . 99',
  'source': 'amazon',
  'cluster_id': 529})

(0.19961591,
 {'id': 818,
  'description': "canopus ( 770 - 10158 - 100 ) let ' s edit 2 . 0",
  'manufacturer': 'canopus',
  'price': '99',
  'name': "canopus 77010158100 let ' s edit",
  'source': 'google',
  'cluster_id': 818},
 {'id': 4046,
  'name': "canopus let ' s edit - video editing software - win consumer video editing software",
  'description': "let ' s edit - realtime video editing software with movie - style effects - win canopus let ' s edit is fast and easy - to - use video",
  'manufacturer': '',
  'price': '99 . 95',
  'source': 'amazon',
  'cluster_id': 818})

(0.2615306,
 {'id': 152,
  'description': 'pdf create ! 4 . 0 us en retail',
  'manufacturer': 'nuance communications inc .',
  'price': '49 . 99',
  'name': 'pdf create ! 4 . 0 retail',
  'source': 'google',
  'cluster_id': 152},
 {'id': 3840,
  'name': 'nuance communications inc . - m009a - l00 - 4 . 0 - pdf create v4 english',
  'description': 'pdf create ! 4 enables users to create pdf documents that are compact and easy to e - mail are universally viewable and print just as they would in the',
  'manufacturer': '',
  'price': '46 . 42',
  'source': 'amazon',
  'cluster_id': 152})

(0.26023877,
 {'id': 662,
  'description': 'terminal server lets you deliver windows - based applications or the windows desktop itself to virtually any computing device -- including those that cannot run windows . terminal server provides',
  'manufacturer': 'microsoft',
  'price': '669',
  'name': 'microsoft windows terminal server 2003 client additional license for users - 5 user',
  'source': 'google',
  'cluster_id': 662},
 {'id': 2908,
  'name': 'win 2003 ter svr cal 5pk microsoft r19 - 00846',
  'description': 'model - ms28736nt vendor - microsoft features - windows terminal svr cal 2003 english microsoft license pack user 5 pack calmanufacturer warranty : 90 days',
  'manufacturer': '',
  'price': '762 . 95',
  'source': 'amazon',
  'cluster_id': 662})

In [47]:
test_left_vector_dict, test_right_vector_dict = \
    test_multisig_dict[('manufacturer',)]

for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.98011965,
 {'id': 1009,
  'description': 'test and improve your memory is a personal trainer for the mind . for your brain to operate it is peak it needs regular exercise . keep it in shape',
  'manufacturer': 'tri synergy',
  'price': '29 . 99',
  'name': 'test and improve your memory ( dvd box )',
  'source': 'google',
  'cluster_id': 1009},
 {'id': 2180,
  'name': 'tri synergy inc test & improve your memory - scientific brain training',
  'description': 'engage challenge and train your memory as it is a crucial area of the brain that is important to your life . includes many games to have fun with various',
  'manufacturer': '',
  'price': '24 . 31',
  'source': 'amazon',
  'cluster_id': 1009})

(0.96931344,
 {'id': 1183,
  'description': 'create the perfect pet for your sims to train play with and love throughout a lifetime . pick everything from their paws to their personality . choose from dozens of',
  'manufacturer': 'aspyr media',
  'price': '34 . 99',
  'name': 'sims 2 pets expansion pack',
  'source': 'google',
  'cluster_id': 1183},
 {'id': 2255,
  'name': 'sims 2 pets for mac',
  'description': 'system requirements : requires the full version of the sims 2 for mac os x to play operating system : mac os x 10 . 3 . 9 or later',
  'manufacturer': '',
  'price': '34 . 99',
  'source': 'amazon',
  'cluster_id': 1183})

(0.9664032,
 {'id': 918,
  'description': 'mobi 70008 recam 4 . 6 camera monitoring and notification software surveillance software captures and analyzes images for advanced monitoring and recording of home or office ; program performs simultaneous',
  'manufacturer': 'mobi technologies inc .',
  'price': '129',
  'name': 'recam remote monitoring software',
  'source': 'google',
  'cluster_id': 918},
 {'id': 3921,
  'name': 'mobi - cam 70008 monitoring and notification software',
  'description': 'easily set - up and monitor your home or business observation system from anywhere notification of alarm and viewing via e - mail internet or cell phone multiple camera support',
  'manufacturer': 'mobi - cam',
  'price': '89 . 69',
  'source': 'amazon',
  'cluster_id': 918})

(0.9550617,
 {'id': 845,
  'description': "iplaymusic ' s beginner guitar lessons is the first guitar learning program optimized for the mac and video ipod . the software takes advantage of ilife applications resident on today",
  'manufacturer': 'iplaymusic',
  'price': '49 . 99',
  'name': 'iplaymusic beginner guitar lessons for the mac and ipod',
  'source': 'google',
  'cluster_id': 845},
 {'id': 2324,
  'name': "wingnuts 2 : raina ' s revenge",
  'description': 'system requirements : mac os x 10 . 4 + g4 / g5 / intel 800 + mhz cpu 512 mb ram 32 mb video card 950 mb hard drive',
  'manufacturer': '',
  'price': '28 . 99',
  'source': 'amazon',
  'cluster_id': 845})

(0.9693135,
 {'id': 641,
  'description': "sims 2 : nightlife takes your sim into the night . explore all of your favorite after - dark activities as you discover your sims ' love lives or have",
  'manufacturer': 'aspyr media',
  'price': '34 . 99',
  'name': 'sims 2 nightlife expansion pack',
  'source': 'google',
  'cluster_id': 641},
 {'id': 1908,
  'name': 'aspyr media inc sims 2 nightlife',
  'description': "send your sims on an epic night out ! your sims are on the town hitting all the swanky hot spots . whether they ' re dancing until dawn romancing",
  'manufacturer': '',
  'price': '33 . 21',
  'source': 'amazon',
  'cluster_id': 641})

(0.9720139,
 {'id': 1124,
  'description': 'printmusic ! is simply the best entry level notation software . consider it a word processor for printing music notes instead of text . designed for the less demanding user',
  'manufacturer': 'makemusic !',
  'price': '69 . 99',
  'name': 'emedia print music 2006 win / mac',
  'source': 'google',
  'cluster_id': 1124},
 {'id': 3546,
  'name': 'make printmusic 2006 software music production software',
  'description': "printmusic 2006 - sheet music creation software - mac os x and windows designed for educators worship directors performing musicians composers and arrangers who don ' t require the advanced",
  'manufacturer': '',
  'price': '75 . 95',
  'source': 'amazon',
  'cluster_id': 1124})

(0.9858687,
 {'id': 529,
  'description': "zoo tycoon 2 : marine mania is a great new expansion where you ' ll help your zoo make a big splash ! add killer whales manta rays and other",
  'manufacturer': 'microsoft',
  'price': '19 . 99',
  'name': 'zoo tycoon 2 : marine mania expansion',
  'source': 'google',
  'cluster_id': 529},
 {'id': 2963,
  'name': 'zoo tycoon for windows',
  'description': 'everybody likes the zoo . and why not casual walks exotic animals and fried foods are good fun . managing a zoo of your very own is even better .',
  'manufacturer': '',
  'price': '25 . 99',
  'source': 'amazon',
  'cluster_id': 529})

(0.9728209,
 {'id': 818,
  'description': "canopus ( 770 - 10158 - 100 ) let ' s edit 2 . 0",
  'manufacturer': 'canopus',
  'price': '99',
  'name': "canopus 77010158100 let ' s edit",
  'source': 'google',
  'cluster_id': 818},
 {'id': 4046,
  'name': "canopus let ' s edit - video editing software - win consumer video editing software",
  'description': "let ' s edit - realtime video editing software with movie - style effects - win canopus let ' s edit is fast and easy - to - use video",
  'manufacturer': '',
  'price': '99 . 95',
  'source': 'amazon',
  'cluster_id': 818})

(0.96419114,
 {'id': 152,
  'description': 'pdf create ! 4 . 0 us en retail',
  'manufacturer': 'nuance communications inc .',
  'price': '49 . 99',
  'name': 'pdf create ! 4 . 0 retail',
  'source': 'google',
  'cluster_id': 152},
 {'id': 3840,
  'name': 'nuance communications inc . - m009a - l00 - 4 . 0 - pdf create v4 english',
  'description': 'pdf create ! 4 enables users to create pdf documents that are compact and easy to e - mail are universally viewable and print just as they would in the',
  'manufacturer': '',
  'price': '46 . 42',
  'source': 'amazon',
  'cluster_id': 152})

(0.9858687,
 {'id': 662,
  'description': 'terminal server lets you deliver windows - based applications or the windows desktop itself to virtually any computing device -- including those that cannot run windows . terminal server provides',
  'manufacturer': 'microsoft',
  'price': '669',
  'name': 'microsoft windows terminal server 2003 client additional license for users - 5 user',
  'source': 'google',
  'cluster_id': 662},
 {'id': 2908,
  'name': 'win 2003 ter svr cal 5pk microsoft r19 - 00846',
  'description': 'model - ms28736nt vendor - microsoft features - windows terminal svr cal 2003 english microsoft license pack user 5 pack calmanufacturer warranty : 90 days',
  'manufacturer': '',
  'price': '762 . 95',
  'source': 'amazon',
  'cluster_id': 662})